In [3]:
import vectorbt as vbt
import pandas as pd

In [7]:
# Carregando dados de entrada
df = pd.read_csv('data/WINM24_05_MIN.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df.head()

,open,high,low,close,tick_volume,spread,real_volume
datetime,,,,,,,
2024-01-02 12:20:00,138700.0,138700.0,138700.0,138700.0,1,5,1
2024-01-02 12:25:00,138550.0,138835.0,138500.0,138500.0,5,5,7
2024-01-02 12:40:00,138350.0,138350.0,138350.0,138350.0,1,5,1
2024-01-02 12:55:00,138310.0,138310.0,138310.0,138310.0,13,5,13
2024-01-02 14:40:00,138600.0,138600.0,138600.0,138600.0,3,5,3


In [9]:
# Estratégia: Comprar quando o preço de fechamento cruza a banda inferior e vender quando cruza a banda superior
# Relação risco X Retirno (2:1, 1:1, 1:2, 1:3)

bb = vbt.BBANDS.run(df['close'], window=20)
entries = df['close'] < bb.lower
exits = df['close'] > bb.upper

risk_reward_ratios = [(1, 1), (1, 2), (1, 3)]
stats_df = pd.DataFrame()

stats_df = pd.DataFrame(columns=[
    'Relação de Risco', 'Total de Trades', 'Win Rate', 'Avg Winning Trade', 
    'Avg Losing Trade', 'Total Return'
])

for sl_ratio, tp_ratio in risk_reward_ratios:
    sl_stop = 0.2 * sl_ratio
    tp_stop = 2 * tp_ratio

    pf = vbt.Portfolio.from_signals(
        df['close'], entries, exits, freq='5min', 
        sl_stop=sl_stop, tp_stop=tp_stop, init_cash=10000
    )

    stats = pf.stats()
    print(stats)
    temp = pd.DataFrame([{
        'Relação de Risco': f'{sl_ratio}:{tp_ratio}',
        'Total de Trades': stats['Total Trades'],
        'Win Rate': stats['Win Rate [%]'],
        'Avg Winning Trade': stats['Avg Winning Trade [%]'],
        'Avg Losing Trade': stats['Avg Losing Trade [%]'],
        'Max Drawdown': stats['Max Drawdown [%]'],
        'Total Return': stats['Total Return [%]']
    }])
    stats_df = pd.concat([stats_df, temp])
stats_df


Start                               2024-01-02 12:20:00
End                                 2024-05-17 18:20:00
Period                                 25 days 07:35:00
Start Value                                     10000.0
End Value                                  10359.340449
Total Return [%]                               3.593404
Benchmark Return [%]                          -6.849315
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               4.426768
Max Drawdown Duration                  20 days 20:10:00
Total Trades                                         86
Total Closed Trades                                  86
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  69.767442
Best Trade [%]                                 1.251522
Worst Trade [%]                               -1

C:\Users\55869\AppData\Local\Temp\ipykernel_27340\1152362243.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats_df = pd.concat([stats_df, temp])


Start                               2024-01-02 12:20:00
End                                 2024-05-17 18:20:00
Period                                 25 days 07:35:00
Start Value                                     10000.0
End Value                                  10359.340449
Total Return [%]                               3.593404
Benchmark Return [%]                          -6.849315
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               4.426768
Max Drawdown Duration                  20 days 20:10:00
Total Trades                                         86
Total Closed Trades                                  86
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  69.767442
Best Trade [%]                                 1.251522
Worst Trade [%]                               -1

,Relação de Risco,Total de Trades,Win Rate,Avg Winning Trade,Avg Losing Trade,Total Return,Max Drawdown
0,1:1,86,69.767442,0.328612,-0.617398,3.593404,4.426768
0,1:2,86,69.767442,0.328612,-0.617398,3.593404,4.426768
0,1:3,86,69.767442,0.328612,-0.617398,3.593404,4.426768
